# Natural Language Processing

## Cómo importar las librerías


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importar el data set


In [3]:
dataset = pd.read_csv("Restaurant_Reviews.tsv", delimiter = "\t", quoting = 3)

## Limpieza de texto

In [4]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 1000):
    review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
    review = review.lower().split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package stopwords to /home/flavio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Crear el Bag of Words

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, 1].values

## Dividir el data set en conjunto de entrenamiento y conjunto de testing

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

## Ejercicio

Ejecuta los otros algoritmos de clasificación que elaboramos en la Parte 3 - Clasificación, además del que ya he usado en esta última clase de la sección.

### Instrucciones de tareas

Lee los enunciados e intenta hacer los ejercicios. Envía tu resultado de cada uno de ellos y tanto nosotros como los otros estudiantes te daremos feedback. El objetivo es intentar adivinar cuál de todos los modelos que vas a aplicar durante esta sección nos da el mejor resultado de clasificación de las valoraciones del restaurante. Así que a por ello!

### Preguntas de esta tarea

Ejecuta los otros algoritmos de clasificación que elaboramos en la Parte 3 - Clasificación, además del que ya he usado en esta última clase de la sección.

Evalúa la eficacia de todos estos modelos. \
Intenta mejorar la Accuracy (exactitud) obtenida en la clase anterior. Recuerda también que la Accuracy no es suficiente, también hay que mirar otras métricas como por ejemplo

- Accuracy, la exactitud de la predicción: (TP+TN)/(TP+TN+FP+FN)
- Precision, medida de la precisión del algoritmo para la clase positiva: TP/(TP+FP)
- Recall, medida de la completitud del algoritmo: TP/(TP+FN)
- F1 Score, compromiso entre la precisión y la completitud: 2*Precision*Recall/(Precision+Recall)

Os recuerdo que TP = #Verdaderos Positivos, TN =#Verdaderos Negativos, FP = #Falsos Positivos y FN = #Falsos Negativos

Intenta otros algoritmos de clasificación que no hemos cubierto en la Parte 3 - Clasificación. Los buenos para NLP incluyen:

- CART
- C5.0
- Máxima Entropía


## Algoritmos de clasificación

In [33]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

def obtener_resultados(y_test, y_pred):
    # Elaborar una matriz de confusión
    cm = confusion_matrix(y_test, y_pred)
    TP = cm[0][0]
    TN = cm[1][1]
    FP = cm[0][1]
    FN = cm[1][0]
    Total = TP+TN+FP+FN
    result = { 'Accuracy': round((TP+TN)/Total,4) , 'Precision': round(TP/(TP+FP),4), 'Recall': round(TP/(TP+FN),2) }
    result['F1Score'] = round( 2*result['Precision']*result['Recall']/(result['Precision']+result['Recall']),4)
    result['Matriz'] = cm
    return result

def logisticRegression(X_train, y_train, X_test):    
    classifier = LogisticRegression(random_state = 0)
    classifier.fit(X_train, y_train)
    return classifier.predict(X_test)

def neighborsClassifier(X_train, y_train, X_test):    
    classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    classifier.fit(X_train, y_train)
    return classifier.predict(X_test)

def kernelSvcRbf(X_train, y_train, X_test):
    classifier = SVC(kernel = "rbf", random_state = 0)
    classifier.fit(X_train, y_train)
    return classifier.predict(X_test)

def kernelSvcLinear(X_train, y_train, X_test):
    classifier = SVC(kernel = "linear", random_state = 0)
    classifier.fit(X_train, y_train)
    return classifier.predict(X_test)

def naiveBayes(X_train, y_train, X_test):
    classifier = GaussianNB()
    classifier.fit(X_train, y_train)
    return classifier.predict(X_test)

def decisionTree(X_train, y_train, X_test):   
    classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
    classifier.fit(X_train, y_train)
    return classifier.predict(X_test)

def randomForest(X_train, y_train, X_test):    
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    classifier.fit(X_train, y_train)
    return classifier.predict(X_test)

def decisionTreeClassifierGini(X_train, y_train, X_test):    
    classifier = DecisionTreeClassifier(criterion = 'gini',max_depth=None, random_state = 0)
    classifier.fit(X_train, y_train)
    return classifier.predict(X_test)

def maxEntropies(X_train, y_train, X_test):    
    classifier = LogisticRegression(penalty='l2', C=1, solver='lbfgs', multi_class='multinomial', max_iter=1000, random_state=42)
    classifier.fit(X_train, y_train)
    return classifier.predict(X_test)

## Cálculos por cada algoritmo

In [34]:
results = []
y_pred = logisticRegression(X_train, y_train, X_test)
result = obtener_resultados(y_test, y_pred)
result['Model'] = 'LogisticRegression'
results.append(result)
y_pred = neighborsClassifier(X_train, y_train, X_test)
result = obtener_resultados(y_test, y_pred)
result['Model'] = 'KNeighborsClassifier'
y_pred = kernelSvcRbf(X_train, y_train, X_test)
result = obtener_resultados(y_test, y_pred)
result['Model'] = 'SVC RBF'
results.append(result)
y_pred = kernelSvcLinear(X_train, y_train, X_test)
result = obtener_resultados(y_test, y_pred)
result['Model'] = 'SVC Linear'
results.append(result)
y_pred = naiveBayes(X_train, y_train, X_test)
result = obtener_resultados(y_test, y_pred)
result['Model'] = 'GaussianNB'
results.append(result)
y_pred = decisionTree(X_train, y_train, X_test)
result = obtener_resultados(y_test, y_pred)
result['Model'] = 'DecisionTree'
results.append(result)
y_pred = randomForest(X_train, y_train, X_test)
result = obtener_resultados(y_test, y_pred)
result['Model'] = 'RandomForest'
results.append(result)
y_pred = decisionTreeClassifierGini(X_train, y_train, X_test)
result = obtener_resultados(y_test, y_pred)
result['Model'] = 'DecisionTreeGini'
results.append(result)
y_pred = maxEntropies(X_train, y_train, X_test)
result = obtener_resultados(y_test, y_pred)
result['Model'] = 'MaxEntropies'
results.append(result)

## Visualización de los resultados

In [35]:
dfResult = pd.DataFrame(results)
dfResult = dfResult[['Model', 'Accuracy', 'Precision', 'Recall', 'F1Score', 'Matriz']]
print(dfResult)

                Model  Accuracy  Precision  Recall  F1Score  \
0  LogisticRegression     0.715     0.7835    0.68   0.7281   
1             SVC RBF     0.730     0.9278    0.66   0.7713   
2          SVC Linear     0.715     0.7526    0.69   0.7199   
3          GaussianNB     0.730     0.5670    0.82   0.6704   
4        DecisionTree     0.705     0.7732    0.67   0.7179   
5        RandomForest     0.705     0.8763    0.64   0.7397   
6    DecisionTreeGini     0.685     0.7526    0.65   0.6975   
7        MaxEntropies     0.720     0.7732    0.69   0.7292   

                 Matriz  
0  [[76, 21], [36, 67]]  
1   [[90, 7], [47, 56]]  
2  [[73, 24], [33, 70]]  
3  [[55, 42], [12, 91]]  
4  [[75, 22], [37, 66]]  
5  [[85, 12], [47, 56]]  
6  [[73, 24], [39, 64]]  
7  [[75, 22], [34, 69]]  
